## Multiple Basis Regression

#### Derek Burmaster
#### PhD Student, Flow Assurance Research Group
#### Hildebrand Department of Petroleum and Geosystems Engineering, Cockrell School of Engineering
##### [LinkedIn](www.linkedin.com/in/derek-burmaster)

### Subsurface Machine Learning Course, The University of Texas at Austin
#### Hildebrand Department of Petroleum and Geosystems Engineering, Cockrell School of Engineering
#### Department of Geological Sciences, Jackson School of Geosciences

_____________________

Workflow supervision and review by:

#### Instructor: Prof. Michael Pyrcz, Ph.D., P.Eng., Associate Professor, The Univeristy of Texas at Austin
##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Course TA: Ademide Mabadeje, Graduate Student, The University of Texas at Austin
##### [LinkedIn](https://www.linkedin.com/in/ademidemabadeje/)


### Executive Summary

Regression is used to fit trends to data in all sorts of applications, but the trend is almost always based on a single function. This highly interactive workflow makes it possible to use a combination of functions as the basis for regression, and can fit trends to generated data, or to data provided by the user.

### Import Packages
The following packages are used in the workflow

In [1]:
import numpy as np                                        # for working with data and model arraysimport pandas as pd
from ipywidgets import widgets                            # for sliders, checkboxes, and other user inputs
import pandas as pd                                       # for working with tabular data (DataFrames)
import matplotlib.pyplot as plt                           # for plotting
from sklearn.linear_model import LinearRegression         # for performing final regression
import asyncio                                            # for asynchronous scripting
import io                                                 # for converting .csv files to and from text...

### Functions

The following basic functions will be used in the workflow. Several main functions are listed in the main body of the workflow, rather than in this section, as they from the main structure of the workflow, and are only presented in function form to allow for their use in dashboards via ipywidgets.

In [2]:
def data_generator(Poly_neg5= 0.0, Poly_neg4= 0.0, Poly_neg3= 0.0, Poly_neg2= 0.0, Poly_neg1= 0.0,
    Poly_0= -7.0, Poly_1= 2.0, Poly_2= 0.0, Poly_3= 0.0, Poly_4= 0.0, Poly_5= 0.0,
    Exponential_lead= 0.0, Exponential_internal= 1.0, ln_lead= 0.0, ln_internal= 1.0,
    noise_seed = 7, noise_std_dev = 3,
    y_min = -10, y_max = 10,):    # Generates a dataframe with data based on the provided function parameters
    
    function_coeffs = {'Poly_neg5': Poly_neg5, 'Poly_neg4': Poly_neg4, 'Poly_neg3': Poly_neg3, 'Poly_neg2': Poly_neg2, 'Poly_neg1': Poly_neg1,
        'Poly_0': Poly_0, 'Poly_1': Poly_1, 'Poly_2': Poly_2, 'Poly_3': Poly_3, 'Poly_4': Poly_4, 'Poly_5': Poly_5,
        'Exponential_lead': Exponential_lead, 'Exponential_internal': Exponential_internal, 'ln_lead': ln_lead, 'ln_internal': ln_internal}
    
    function_printer(**function_coeffs)

    # Domain Paramaters
    x_min = 1
    x_max = 10
    space_length = 99

    # Generate domain and noise
    input_data = np.linspace(x_min,x_max,space_length)
    np.random.seed(seed = noise_seed)
    noise = np.random.normal(0,noise_std_dev,space_length)

    # Calculate function values
    df = pd.DataFrame()
    df['x'] = input_data
    df['y_clean'] = function_calc(df['x'],**function_coeffs)
    
    # Add noise
    df['noise'] = noise
    df['y'] = function_calc(df['x'],**function_coeffs)+df['noise']
    df['y'] = df['y_clean'] + df['noise']
    
    # Plot function values, with and without noise
    plt.subplot(111)
    plt.plot(df['x'], df['y'], 'o', label='Function Data with noise', color = 'red', alpha = 0.5, markeredgecolor = 'black')
    plt.plot(df['x'], df['y_clean'], 'o', label='Function Data without noise', color = 'blue', alpha = 0.2, markeredgecolor = 'black')
    plt.title('Function Values')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.ylim(y_min,y_max)
    plt.legend()
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.0, wspace=0.2, hspace=0.2)
    plt.show()
    
    # Store data to .csv for use outside this function
    df.to_csv('multiple_basis_regression_data.csv')



def function_calc(x,**function_coeffs):    # Calculates an output given an input and function parameters
    
    output = (function_coeffs.get('Poly_neg5')*x**-5+function_coeffs.get('Poly_neg4')*x**-4+function_coeffs.get('Poly_neg3')*x**-3+function_coeffs.get('Poly_neg2')*x**-2+function_coeffs.get('Poly_neg1')*x**-1+                   
        function_coeffs.get('Poly_0')*x**0+function_coeffs.get('Poly_1')*x**1+function_coeffs.get('Poly_2')*x**2+function_coeffs.get('Poly_3')*x**3+function_coeffs.get('Poly_4')*x**4+function_coeffs.get('Poly_5')*x**5+
        function_coeffs.get('Exponential_lead')*np.exp(function_coeffs.get('Exponential_internal')*x)+
        function_coeffs.get('ln_lead')*np.log(function_coeffs.get('ln_internal')*x))
    
    return(output)



def function_printer(**function_coeffs):    # Prints a function, given the function's parameters  
    
    function = 'f(x) = '
    
    for coeff in function_coeffs:    
        
        coefficient = function_coeffs.get(coeff)    # Gets parameter value from dictionary
        if coefficient == 1:
            coefficient = ''    # Avoids 1*x -style coefficients
        else:
            coefficient = f'({coefficient})*'
        
        if coeff.startswith('Poly_neg'):    # Checks for negative polynomial parameters                           
            if coefficient != '(0.0)*':
                exponent = coeff.lstrip('Poly_neg')
                if function != 'f(x) = ':
                    function = f'{function} + '
                function = f'{function}{coefficient}x^(-{exponent})'
            
        elif coeff.startswith('Poly_'):    # Checks for non-negative polynomial parameters
            if coefficient != '(0.0)*':
                exponent = coeff.lstrip('Poly_')
                if function != 'f(x) = ':
                    function = f'{function} + '
                function = f'{function}{coefficient}x^({exponent})'
        
        elif coeff.startswith('Exponential_'):    # Checks for exponential parameters
            if coefficient != 0:
                if coeff.endswith('_lead'):
                    lead_coefficient = coefficient
        
                elif coeff.endswith('_internal'):
                    if lead_coefficient != '(0.0)*':
                        if function != 'f(x) = ':
                            function = f'{function} + '
                        function = f'{function}{lead_coefficient}e^({coefficient}x)'
                    
        elif coeff.startswith('ln_'):    # Checks for exponential parameters
            if coefficient != 0:
                if coeff.endswith('_lead'):
                    lead_coefficient = coefficient
        
                elif coeff.endswith('_internal'):
                    if lead_coefficient != '(0.0)*':
                        if function != 'f(x) = ':
                            function = f'{function} + '
                        function = f'{function}{lead_coefficient}ln({coefficient}x)'
                        
    
    if function == 'f(x) = ':    # if no function values have been written,
        function = 'f(x) = 0'    # Sets function = 0
    print(function)
    
    

def build_regression_space(Poly_neg5_basis_checkbox, Poly_neg4_basis_checkbox, Poly_neg3_basis_checkbox, Poly_neg2_basis_checkbox, Poly_neg1_basis_checkbox,
                    Poly_0_basis_checkbox, Poly_1_basis_checkbox, Poly_2_basis_checkbox, Poly_3_basis_checkbox, Poly_4_basis_checkbox, Poly_5_basis_checkbox,
                    Exponential_basis_checkbox,ln_basis_checkbox):
    
    # Combine basis booleans into dictionary for easy access/calling
    basis_dict = { 'Poly_neg5_basis': Poly_neg5_basis_checkbox, 'Poly_neg4_basis': Poly_neg4_basis_checkbox, 'Poly_neg3_basis': Poly_neg3_basis_checkbox, 'Poly_neg2_basis': Poly_neg2_basis_checkbox, 'Poly_neg1_basis': Poly_neg1_basis_checkbox,
        'Poly_0_basis': Poly_0_basis_checkbox, 'Poly_1_basis': Poly_1_basis_checkbox, 'Poly_2_basis': Poly_2_basis_checkbox, 'Poly_3_basis': Poly_3_basis_checkbox, 'Poly_4_basis': Poly_4_basis_checkbox, 'Poly_5_basis': Poly_5_basis_checkbox,
        'Exponential_basis': Exponential_basis_checkbox,
        'ln_basis': ln_basis_checkbox}
    
    # Call data from csv created in previous dashboard step
    df = pd.read_csv('multiple_basis_regression_data.csv')
    
    # Declare new dataframe for basis space
    df_basis_space = pd.DataFrame()
    
    # Perform basis calculations
    for basis in basis_dict:
        if basis_dict.get(basis) == True:

            # Polynomial basis calculations
            if basis.startswith('Poly_'):
                if basis.startswith('Poly_neg'):
                    exponent = f'-{basis.lstrip("Poly_neg").rstrip("_basis")}'
                else:
                    exponent = basis.lstrip('Poly_').rstrip('_basis')
                df_basis_space[f'x^{exponent}'] = df['x']**float(exponent)

            # Exponential basis calculation
            if basis == 'Exponential_basis':
                df_basis_space['e^x'] = np.exp(df['x'])

            # Natural Logarithmic basis calculation
            if basis == 'ln_basis':
                df_basis_space['ln(x)'] = np.log(df['x'])
                
    return(df_basis_space)


def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

### Multiple Basis Regression

##### Dashboard 1: Data
* Provide checkbox for data upload
* Collect Data
    * If data will be generated:
        * Provide sliders for each possible parent function
        * Provide slider for noise
        * Provide sliders for plot scaling
        * Run data generator
            * Print the user-generated function
            * Generate dataframe, domain values
            * Generate function values
            * Add noise
            * Plot data, both original and noisy        
    * If data will be uploaded:
        * Provide file upload widget
* Save data to csv, for use in Dashboard 2

##### Dashboard 2: Regression
* Provide checkboxes for each basis function
* Call data from previous step
* Build the basis spaces chosen
* Run the regression
* Plot the both sets of data again, this time with the model

### Dashboard 1. Data

This workflow is designed for both education and general use. As such it can generate data based on a user-defined function, or it can accept data from a .csv file. The workflow defaults to generating data.

The dashboard generated by the next code block allows the user to input data, either by uploading a dataset or by generating data from a combination of various parent functions. It also allows for the addition of noise, and chaning the scale of the plot. 

The dashboard presents its interpretation of the function values as given, and a plot of the function values with an without any noise.

In [4]:
# Get asyncio started
%gui asyncio

# Build data source selection widget
data_source_buttons = widgets.RadioButtons(
    options=['Generate', 'Upload'],
    value='Generate',
    description='Data source:',
    disabled=False
)

# Define main dashboard function
def get_data(data_source_buttons):
    
    df_upload_case = pd.DataFrame()
    
    if data_source_buttons == 'Generate' :
        df_upload_case.at[0,'upload_data']= True       
        
        # Build function parameter input widgets
        Poly_neg5_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(x^{-5}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_neg4_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(x^{-4}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_neg3_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(x^{-3}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_neg2_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(x^{-2}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_neg1_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(x^{-1}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_0_slider= widgets.FloatSlider(value = -7, min = -10,max = 10, step = 0.1, description = r"\(x^{0}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_1_slider= widgets.FloatSlider(value = 2, min = -10,max = 10, step = 0.1, description = r"\(x^{1}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_2_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(x^{2}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_3_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(x^{3}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_4_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(x^{4}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Poly_5_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(x^{5}\)",
            layout=widgets.Layout(width='30px', height='200px'), orientation='vertical',continuous_update=False)
        Exponential_lead_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(a*e^x\)",
            layout=widgets.Layout(width='60px', height='200px'), orientation='vertical',continuous_update=False)
        Exponential_internal_slider= widgets.FloatSlider(value = 1, min = -10,max = 10, step = 0.1, description = r"\(e^{b*x}\)",
            layout=widgets.Layout(width='60px', height='200px'), orientation='vertical',continuous_update=False)
        ln_lead_slider= widgets.FloatSlider(value = 0, min = -10,max = 10, step = 0.1, description = r"\(a*lnx\)",
            layout=widgets.Layout(width='60px', height='200px'), orientation='vertical',continuous_update=False)
        ln_internal_slider= widgets.FloatSlider(value = 1, min = 0.1,max = 10, step = 0.1, description = r"\(ln(b*x)\)",
            layout=widgets.Layout(width='60px', height='200px'), orientation='vertical',continuous_update=False)
        
        # Build noise generation widgets
        noise_std_dev_slider= widgets.FloatSlider(value = 2, min = 0,max = 10, step = 0.1, description = "noise std dev",continuous_update=False)
        noise_seed_slider= widgets.IntSlider(value = 7, min = 0,max = 100, step = 1, description = "noise seed",continuous_update=False)

        # Build graph parameter input widgets
        y_min_slider= widgets.FloatSlider(value = -10, min = -1000,max = 900, step = 10, description = "y-axis min",continuous_update=False)
        y_max_slider= widgets.FloatSlider(value = 10, min = -900,max = 1_000, step = 10, description = "y-axis max",continuous_update=False)

        # Format sliders into rows with headers (d = dash, r = row; i.e. d1r2 = dash 1, row 2)
        d1r1_header = widgets.Text(value='Parent function parameters',layout=widgets.Layout(width='200px', height='30px'))
        d1r1 = widgets.HBox([Poly_neg5_slider,Poly_neg4_slider,Poly_neg3_slider,Poly_neg2_slider,Poly_neg1_slider,Poly_0_slider,
            Poly_1_slider,Poly_2_slider,Poly_3_slider,Poly_4_slider,Poly_5_slider,
            Exponential_lead_slider,Exponential_internal_slider,ln_lead_slider,ln_internal_slider],)
        d1r2_header = widgets.Text(value='Noise parameters',layout=widgets.Layout(width='200px', height='30px'))
        d1r2 = widgets.HBox([noise_std_dev_slider,noise_seed_slider],)
        d1r3_header = widgets.Text(value='Plot limits',layout=widgets.Layout(width='200px', height='30px'))
        d1r3 = widgets.HBox([y_min_slider,y_max_slider,],)
        
        # Compile rows into a single user interface
        d1_ui = widgets.VBox([d1r1_header,d1r1,d1r2_header,d1r2,d1r3_header,d1r3],)

        # Build generator sub-dashboard
        generator_dash = widgets.interactive_output(data_generator, {'Poly_neg5' : Poly_neg5_slider, 'Poly_neg4' : Poly_neg4_slider, 'Poly_neg3' : Poly_neg3_slider, 'Poly_neg2' : Poly_neg2_slider, 'Poly_neg1' : Poly_neg1_slider,
            'Poly_0' : Poly_0_slider, 'Poly_1' : Poly_1_slider, 'Poly_2' : Poly_2_slider, 'Poly_3' : Poly_3_slider, 'Poly_4' : Poly_4_slider, 'Poly_5' : Poly_5_slider,
            'Exponential_lead' : Exponential_lead_slider, 'Exponential_internal' : Exponential_internal_slider, 'ln_lead' : ln_lead_slider, 'ln_internal' : ln_internal_slider,
            'noise_std_dev' : noise_std_dev_slider, 'noise_seed' : noise_seed_slider,
            'y_min' : y_min_slider, 'y_max' : y_max_slider,})
        
        # Display data generator sub-dashboard
        display(d1_ui,generator_dash)
        
        
    elif data_source_buttons == 'Upload':
        df_upload_case.at[0,'upload_data']= False
        
        # Build file upload widget
        uploader = widgets.FileUpload()
        

        # Create background loop to update file after receiving user input
        async def background_loop():
            df = pd.DataFrame()
            while True:
                x = await wait_for_change(uploader, 'value')
                input_file = list(uploader.value.values())[0]
                content = input_file['content']
                content = io.StringIO(content.decode('utf-8'))
                df = pd.read_csv(content)
                df.to_csv('multiple_basis_regression_data.csv')
        asyncio.ensure_future(background_loop())
        
        # Create file requirement info header
        d1r1_header = widgets.Text(value=
                                   'Please upload a *.csv file,\n with the independent variable column labeled "x", \n and the dependent variable column labeled "y".'
                                   ,layout=widgets.Layout(width='750px', height='30px'))
        
        # Display data upload sub-dashboard
        display(d1r1_header,uploader)

        
#     else:
#         df_upload_case.at[0,'upload_data']= False
        
#         # Base Case goes here    # Base Case goes here    # Base Case goes here    # Base Case goes here
#         print('whoops')
        
    
#     df_upload_case.to_csv('multiple_basis_regression_upload_case.csv') 
        
        
# Generate a header for the Dashboard
d1_header = widgets.VBox([widgets.Text(value='                                                                                                          Dashboard 1: Data',layout=widgets.Layout(width='950px', height='30px'))],)

# Build a box to display the radio buttons in
radio_box = widgets.VBox([data_source_buttons],)

# Build Data Dashboard
dashboard1 = widgets.interactive_output(get_data,{'data_source_buttons':data_source_buttons})

# Display Data Dashboard
display(d1_header,radio_box,dashboard1)

Output()

### Dashboard 2. Regression

This workflow is designed to showcase regression with multiple basis functions. To maximize the utility of the display, the user is able to change the types of functions used in the regression.

The dashboard generated by the next code block allows the user to choose whichever combination of functions they would be interested in.

The dashboard presents a plot of the linear regression model along with thefunction values as shown in the previous dashboard.

In [8]:
def perform_regression(Poly_neg5_basis_checkbox, Poly_neg4_basis_checkbox, Poly_neg3_basis_checkbox, Poly_neg2_basis_checkbox, Poly_neg1_basis_checkbox,
                    Poly_0_basis_checkbox, Poly_1_basis_checkbox, Poly_2_basis_checkbox, Poly_3_basis_checkbox, Poly_4_basis_checkbox, Poly_5_basis_checkbox,
                    Exponential_basis_checkbox, ln_basis_checkbox):

    # Call data from csv created in previous dashboard step
    df = pd.read_csv('multiple_basis_regression_data.csv')
    
    # Read upload case from temporary csv
#     df_upload_case = pd.DataFrame()
#     df_upload_case = pd.read_csv('multiple_basis_regression_upload_case.csv')
#     upload_data = df_upload_case['upload_data'].values[0]   
    
    # Build regression space:
    df_basis_space = build_regression_space(Poly_neg5_basis_checkbox, Poly_neg4_basis_checkbox, Poly_neg3_basis_checkbox, Poly_neg2_basis_checkbox, Poly_neg1_basis_checkbox,
        Poly_0_basis_checkbox, Poly_1_basis_checkbox, Poly_2_basis_checkbox, Poly_3_basis_checkbox, Poly_4_basis_checkbox, Poly_5_basis_checkbox,
        Exponential_basis_checkbox, ln_basis_checkbox,)
    
    # Initialize and fit model
    multi_base_lin_reg = LinearRegression() 
    multi_base_lin_reg.fit(df_basis_space, df['y'])

    # Determine and round MSE
    mse_vs_noise = '%.4g' % ((df['y'] - multi_base_lin_reg.predict(df_basis_space))**2).mean()
    if upload_data == False:
        mse_vs_truth = '%.4g' % ((df['y_clean'] - multi_base_lin_reg.predict(df_basis_space))**2).mean()
    
    # Plot data and model
    plt.subplot(111)
    if upload_data == False:
        plt.plot(df['x'], df['y_clean'], 'o', label='Function values without noise', color = 'blue', alpha = 0.2, markeredgecolor = 'black')
        plt.plot(df['x'], df['y'], 'o', label='Function values with noise', color = 'red', alpha = 0.5, markeredgecolor = 'black')
        plt.plot(df['x'], multi_base_lin_reg.predict(df_basis_space), label=f'Model, MSE = {mse_vs_truth}',color = 'black') 
        plt.title('Function Values versus Model Prediction')
    else:
        plt.plot(df['x'], df['y'], 'o', label='Data Values', color = 'blue', alpha = 0.2, markeredgecolor = 'black')
        plt.plot(df['x'], multi_base_lin_reg.predict(df_basis_space), label=f'Model, MSE = {mse_vs_noise}',color = 'black') 
        plt.title('Data Values versus Model Prediction')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.0, wspace=0.2, hspace=0.2)
    plt.show()
    

# Build modeling dashboard basis checkboxes
Poly_neg5_basis_checkbox = widgets.Checkbox(value=False, description='x^-5')
Poly_neg4_basis_checkbox = widgets.Checkbox(value=False, description='x^-4')
Poly_neg3_basis_checkbox = widgets.Checkbox(value=False, description='x^-3')
Poly_neg2_basis_checkbox = widgets.Checkbox(value=False, description='x^-2')
Poly_neg1_basis_checkbox = widgets.Checkbox(value=False, description='x^-1')
Poly_0_basis_checkbox = widgets.Checkbox(value=True, description='x^0')
Poly_1_basis_checkbox = widgets.Checkbox(value=True, description='x^1')
Poly_2_basis_checkbox = widgets.Checkbox(value=False, description='x^2')
Poly_3_basis_checkbox = widgets.Checkbox(value=False, description='x^3')
Poly_4_basis_checkbox = widgets.Checkbox(value=False, description='x^4')
Poly_5_basis_checkbox = widgets.Checkbox(value=False, description='x^5')
Exponential_basis_checkbox = widgets.Checkbox(value=False, description='ln(x)')
ln_basis_checkbox = widgets.Checkbox(value=False, description='e^x')

# Format checkboxes into a row with a header (d = dash, r = row; i.e. d1r2 = dash 1, row 2)
d2r1_header = widgets.Text(value='Regression basis options',layout=widgets.Layout(width='200px', height='30px'))
d2r1 = widgets.VBox([Poly_neg5_basis_checkbox,Poly_neg4_basis_checkbox,Poly_neg3_basis_checkbox,Poly_neg2_basis_checkbox,Poly_neg1_basis_checkbox],)
d2r2 = widgets.VBox([Poly_0_basis_checkbox,Poly_1_basis_checkbox,Poly_2_basis_checkbox,Poly_3_basis_checkbox,Poly_4_basis_checkbox,Poly_5_basis_checkbox],)
d2r3 = widgets.VBox([Exponential_basis_checkbox,ln_basis_checkbox],)

# Compile rows into a single user interface
d2_ui = widgets.HBox([d2r1,d2r2,d2r3],)

# Run Regression dashboard
dashboard2 = widgets.interactive_output(perform_regression, {'Poly_neg5_basis_checkbox' : Poly_neg5_basis_checkbox, 'Poly_neg4_basis_checkbox' : Poly_neg4_basis_checkbox, 'Poly_neg3_basis_checkbox' : Poly_neg3_basis_checkbox, 'Poly_neg2_basis_checkbox' : Poly_neg2_basis_checkbox, 'Poly_neg1_basis_checkbox' : Poly_neg1_basis_checkbox,
    'Poly_0_basis_checkbox' : Poly_0_basis_checkbox, 'Poly_1_basis_checkbox' : Poly_1_basis_checkbox, 'Poly_2_basis_checkbox' : Poly_2_basis_checkbox, 'Poly_3_basis_checkbox' : Poly_3_basis_checkbox, 'Poly_4_basis_checkbox' : Poly_4_basis_checkbox, 'Poly_5_basis_checkbox' : Poly_5_basis_checkbox,
    'Exponential_basis_checkbox' : Exponential_basis_checkbox, 'ln_basis_checkbox' : ln_basis_checkbox})

# Generate a header for the Dashboard
d2_header = widgets.VBox([widgets.Text(value='                                                                                                          Dashboard 2: Regression',layout=widgets.Layout(width='950px', height='30px'))],)

# Display Regression Dashboard
display(d2_header,d2r1_header,d2_ui,dashboard2)

Text(value='Regression basis options', layout=Layout(height='30px', width='200px'))

Output()

### Remarks

This workflow is designed to showcase regression with multiple basis functions. If you came to this workflow to learn more about regression, I hope that the interactivity of the dashboards has helped you see the benefits of using this type of modeling. If you came to this workflow to do some modeling of your own, I hope things worked perfectly for you. If they didn't, let me know!

I sincerely hope this was helpful,

*Derek Burmaster*

[LinkedIn](https://www.linkedin.com/in/derek-burmaster/)
___________________